# Chapter 5.1 - Text Classification with Logistic Regression

Our first machine-learning challenge is to create a prediction model that categorizes news articles with the appropriate categories from a set of 31 categories: politics, entertainment, etc. The model uses logistic regression techniques in Python on a dataset with headlines, short descriptions, and URLs. 

*NOTE*: Be sure to watch and read the materials posted in the Canvas module before and while you work through this notebook.

**Learning Objectives**

1. Import and run EDA techniques to understand the potential limits and affordances of the dataset with our ML goal in mind.
2. Learn about the basic mechanics of logistic regression (LR).
3. Apply LR to this text classification goal of categorizing the news genre of articles based on potential "features" in the data, such as the article's headline, short description, and URL.

**Sources**

- Notebook modified from Ganesan's LR example exercise: [Text Classification with Logistic Regression](https://github.com/kavgan/nlp-in-practice/tree/master/text-classification)
- Dataset: HuffPost News Dataset in [../data/05-ML](../data/05-ML/news_category_dataset.json)

## Import Libraries

In [ ]:
# You may need to install some of the libraries below
# If so, uncomment any of the below commands
# !pip install matplotlib
# !pip install seaborn
# !pip install mplcyberpunk
# !pip install scikit-learn
# !pip install pickle

In [ ]:
import pandas as pd
import regex as re
import numpy as np
import logging

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import mplcyberpunk
%config InlineBackend.figure_formats = ['svg']

# ML Modeling
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay,roc_curve
from sklearn.preprocessing import normalize
# Saving and importing trained models
import pickle

## 1. Refresher on pandas' dataframes

We can use the pandas library to read in, review, and revise the data set.

Recall how pandas' "panel data" helps us more easily transform multiple types of structured data into what's called a *DataFrame*. Dataframe's are two-dimensional tabular data that are mutable (transformable) with labeled axes (rows and columns). See chapter 3 to refresh your memory, if needed. You can also reference [pandas in general](https://pandas.pydata.org/docs/getting_started/index.html#getting-started), or its [Dataframe datatype](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame).

<figure>
<img src="../images/01_table_dataframe.svg"
     style="max-width: 300px; background: white;"
/>
<figcaption>Panel data "Dataframe" that shows how there are rows for observations and columns for properties of those observations, as well as an index column for easy reference per row.
</figure>

## 2. Import and Review the Data

In [ ]:
 
# Imports the noted JSON file as a pandas DataFrame based on the path below
df = pd.read_json("./../data/05-ML/news_category_dataset.json", lines=True)

### 2.1 General shape and content of the dataset

In [ ]:
df.shape

`.shape` returns a tuple with the panel data 2-dimensional info: 1. the number of rows, and 2. the number of columns.

In [ ]:
df.info()

Looks like there are no `null` values in the dataset.

Let's dig deeper into the columns and values.

### 2.2 Review particular columns

Be sure to review the data per column. The goals of this modeling may be set for you, but you will be asked to perform similar EDA work on your final project, so you can better understand the modeling possibilities and boundaries with your data. So, the below includes a series of exercises for you to understand this dataset before you conduct the actual modeling work.

In [ ]:
# List the columns for reference
df.columns

#### 2.2.1 Describe and examine the `headline` column

We can use `.describe()` on a Series, so we can consider any potential quirks. 

Let's check out the `headline` column, since that's an important column for training our model.

##### What are the summary stats for the `headline`?

In [ ]:
df.headline.describe()

Some initial observations: 

- Noticing how there is a decently sized difference between the `count` and `unique` values
- `top`: How "Sunday Roundup" headline appears 90 times (`freq`) in the `headline` column (`Name: headline, dtype: object`).

Let's review rows with the "Sunday Roundup" value in the headline column by using our new skills with pandas. Below I ...

1. `df.loc[]`: query the dataframe with the location method
2. `[df.headline]`: Specifiy what slice of the data I want to isolate.
3. `.str.contains('Sunday Roundup')`: Since `headline` values are Strings, and I want to isolate any rows with the "Sunday Roundup" headline, search the isolated Series, `headline`,  with the `.str.contains()` method. It takes a string as its main parameter.

In [ ]:
df.loc[df.headline.str.contains('Sunday Roundup')].sample(3)

##### How long are the headlines? (What's the distribution?)

Also curious about the distribution of the length of those headlines, so let's add a new column to the `df` by applying the `len` (length) method to the `headline` column. Below, I do so with the `apply()` method and assign the values per row to a new Series (column) that I call `headline_length`.

In [ ]:
df['headline_length'] = df.headline.apply(len)
df.head()

In [ ]:
'''
  .hist() -- Creates a histogram chart with a dataframe's Series 
  
  Histograms place a metric in bins -- dates in this case -- to understand the distribution of the data. In this case, the distribution of the data over time
'''
df.headline_length.hist(
  figsize=(12,6),
  color='#86bf91',
)

Looks like there's some variance with some headlines that are over 100 characters long and as much as ~300.

Let's use `.describe()` on this new column as a table of values to review.

In [ ]:
df.headline_length.describe()

##### 2.2.1 Exercise -- Observations about the headlines column

- ***ENTER YOUR FIRST OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER YOUR SECOND OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER MORE OBSERVATIONS***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***

#### 2.2.2 What are the range and distribution of dates?

Since we have a `date` column, and the datatype is in a standardized format, we can quickly plot the date range in a histogram figure.

Let's jot down some observations in our notebook.

**NOTE**: Be sure to respond to the questions and add at least one more potential question, observation, and potential explanation for it

##### 2.2.2.1 *EXERCISE* -- Notes on the distribution of the data based on the `date` column

- Articles' publishing dates range between July 2014 and July 2018
  - Question: How might this impact the model's output?
  - ***YOUR RESPONSE HERE***
- Fewer 2018 articles compared to the rest of the dataset.
  - Question: How might this impact the model's output?
  - ***YOUR RESPONSE HERE***

In [ ]:
'''
  .hist() -- Creates a histogram chart with a dataframe's Series 
  
  Histograms place a metric in bins -- dates in this case -- to understand the distribution of the data. In this case, the distribution of the data over time
'''

df.date.hist(
  figsize=(12,6),
  color='#86bf91',
)

#### 2.2.3 Describe and examine the `category` column of news genres in the data

Use *dot notation* and a *column name* with `sample()`` to sample values in that column (Series).

**Remember**: The dataframe stays the same because we are not altering `df`.

In [ ]:
df.category.sample(5)

Now, let's read and understand the news genre categories (politics, entertainment, etc.) in the dataset, as well as their distribution.

##### 2.2.3.1 Examine the `category` counts and values

We can use the `len()` and `set()` functions in Python to isolate the unique number of values in a column. 

`len()` by itself would provide a length of *all values combined*, even if the value is repeated. By using `set()` first on the column, we can tell Python to reduce the column to unique set of values. Then, it will count that unique set with `len()`.

In [ ]:
len(
  set(df['category'].values)
)

In [ ]:
set(
  df['category'].values
)

##### 2.2.3.2 `category` by count

Let's review the distribution of categories, since this could impact the model.

In the code below, we tell Python to:

1. Select the `category` column,
2. Count the values of each value in the column with `value_counts()`,
3. Plot the results with `plot` and provide the value `bar` for its parameter `kind`

##### 2.2.3.3 *EXERCISE* -- Notes on the distribution of the data based on the `category` column

- ***ENTER YOUR FIRST OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER YOUR SECOND OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER MORE OBSERVATIONS***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***

In [ ]:
df['category'].value_counts().plot(kind='bar')

#### 2.2.4 Describe and examine the `short_description` column

In [ ]:
df.short_description.describe()

In [ ]:
df['sd_length'] = df.short_description.apply(len)
df[['short_description','sd_length']].sample(5)

In [ ]:
df.sd_length.describe()

In [ ]:
df.sd_length.hist(
  figsize=(12,6),
  color='#86bf91',
)

##### 2.2.4.1 *EXERCISE* -- Notes on the `sd_length` column

- ***ENTER YOUR FIRST OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER YOUR SECOND OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER MORE OBSERVATIONS***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***

## 3. Process the Data for Classification

Some of the fields will be useful to use for the classification task.

1. `headline`: Since we're writing a model to classify headlines, this column is integral to our goal.
2. `short_description`: This column includes a short descrtiption of the story, which will provide potentially helpful contextual information to make a better model for our genre classifier. We'll see!
3. `tokenized_url`: This column includes information too! Different parts of URLs often capture how people have organized and classified information. So, the URL may also provide potentially helpful contextual information to make a better model. Again, we'll see!

The code below creates 3 new columns for that task.

There's a lot to unpack below, but it basically

1. normalizes the URL for each article via the `tokenize_url()` function, which strips the URL down to the portion where the URL conveys article-specific information. Then, 
2. it creates 3 new columns that are combined with the available data.

In [ ]:
def tokenize_url(url:str):
    url=url.replace("https://www.huffingtonpost.com/entry/","")
    url=re.sub("(\W|_)+"," ",url)
    return url

df['tokenized_url']=df['link'].apply(lambda x:tokenize_url(x))

#just the description
df['text_desc'] = df['short_description']

#description + headline
df['text_desc_headline'] = df['short_description'] + ' '+ df['headline']

#description + headline + tokenized url
df['text_desc_headline_url'] = df['short_description'] + ' '+ df['headline']+" " + df['tokenized_url']

df.info()


Let's sample the new row with the URL info to see what it includes.

##### EXERCISE -- Notes on the on the `text_desc_headline_url` column

- ***ENTER YOUR OBSERVATION HERE***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***
- ***ENTER MORE OBSERVATIONS***
  - Question: How might this impact our model's output?
  - ***YOUR RESPONSE HERE***

In [ ]:
df['text_desc_headline_url'].sample(5).values.tolist()

## 4. Train a Logistic Regression Model

### 4.1 What's a Logistic Regression Model?

Logistic regression returns a probability that something may or may not happen or "be". We can use this probability as a single value to assess the likelihood of the event/being, such as "This X headline is of Y news genre".

So, let's say our LR model returns a value of 0.990 for a particular headline's news genre as being "POLITICS". This probability score is very likely to accurately predict that this headline is indeed an article about POLITICS. Conversely, another headline with a prediction score of 0.005 on that same logistic regression model is very likely not about POLITICS. Yet, what about a headline with a prediction score of 0.6? 

In this lesson, our LR model uses those probability estimates as a binary category. To do so, we must decide what's called a "classification threshold" or "decision threshold". Any value above that threshold indicates a headline is about POLITICS, and any value below the threshold indicates that the headline is not POLITICS, but some other news genre.

The default decision threshold in the scikit-learn code library that we will use is 0.5. But, this library also enables us to "tune" the LR model based on our problem-dependency / context, as well as take the best/top probability score to predict the news genre of the input headline.

### 4.2 How to Train a Drago... Erm ... a LR Model!

#### 4.2.1 *From Words to Numbers!* - Transform a corpus of words into matrix of integers with `CountVectorizer()`

**Before you review the many functions defined in section 4.2 below**, functions that will automate the modeling process, I want you to understand a very important first step: how we get from word label data to numbers for the model. After all, this is "Writing & Digital Media" **:-)**

So, first things first, we're working with text, so strings and characters. But, a logistic regression model requires numbers. So, we need to extract the features of our corpus and transform them into a data type that can be quantified.

`CountVectorizer()` to the rescue!

To better understand how scikit-learn's built-in functions transform the textual data for us, I provide us with a tiny tiny case of a list of sentences, which transforms our textural data into a **sparse matrix** of values that indicate the frequency of times the word (feature/token) is present in a unit of content (document). 

In this tiny case below, our documents are pretty short strings in a list. Those could be much much bigger, and really it should be for this type of modeling. But, for example purposes, let's see how the matrix logs how many instances a word shows up in each document. 

Each row is the document, (i.e., sentence, in this case), while each column is the feature (word/token). So the values at the intersection of the row and column in the matrix report the raw frequency count (integer).

In [ ]:
# Tiny weeny bitty small corpus :-)
list_test_text_1 = ['Hello my name is chris', 'chris this is my python notebook you know python']
# list_test_text_2 = ['Hello my name is chris', 'chris this is my python notebook', 'chris trying to create a big dataset', 'chris is trying different words', 'features of count vectorizer']

# Instantiate an sklearn CountVectorizer (CV) object
test_cv_object = CountVectorizer()

# Use the CV's fit_transform() function to count the frequency of each feature (word/token as columns) in each document (rows)
test_count_matrix = test_cv_object.fit_transform(list_test_text_1)

In [ ]:
# Note the specialized data type for sklearn
type(test_count_matrix)

If I try to print out the sklearn cv matrix, I'll get some basic data structural info.

In [ ]:
test_count_matrix

The matrix has 2 rows (documents/sentences) and 10 columns (words/tokens) with values that are numpy integers.

But, what's the now transformed/quantified data look like? Let's convert this returned sparse matrix as a more recognizable data type: a pandas dataframe.

In [ ]:
# Convert the matrix into a more basic Python array
test_count_array = test_count_matrix.toarray()

# Use that array as the data in the DF, then nicely use CV's `get_feature_names_out()` method to provide the column names
test_cv_df = pd.DataFrame(
  data=test_count_array,
  columns=test_cv_object.get_feature_names_out()
)
test_cv_df

**What do we notice?**

- 2 rows
- 10 columns
- values are the frequency of times a word shows up in a document, such as 
  - `chris`, `is`, and `my` are in both docs
  - `python` shows up twice in the second document
  - Zeros indicate the word doesn't show up in a document

##### Summarize `CountVectorizer()`

In the modeling functions below, the extract_features() function uses CountVectorizer() to, well, extract these features from the much larger corpus. It nicely automates the process for us—writing and calculating at amazing speed and scale. **Can you imagine transforming the data manually?!** I don't think we want to subject ourselves to even such a thought.

### 4.3 Modeling functions

The functions below help us create a systematic and reproducable workflow to train the data.

Be sure to check out my videos that walk through an overview of what they do.

In [ ]:
def _reciprocal_rank(true_genre_labels: list, machine_predicted_genre_labels: list):
    '''
    ## Purpose
    Compute the reciprocal rank at cutoff k

    ## Parameters
        - `true_genre_labels` (List): List of actual news genre labels
        - `machine_predicted_genre_labels` (List): List of news genre labels predicted by the LR algorithm
    
    ## Return Values
        - `recip_rank` (Float): Reciprocal rank
    '''
    
    # add index to list only if machine predicted label exists in true labels
    tp_pos_list = [(idx + 1) for idx, r in enumerate(machine_predicted_genre_labels) if r in true_genre_labels]

    recip_rank = 0
    if len(tp_pos_list) > 0:
        # for reciprocal rank we must find the position of the first **correctly labeled** item
        first_pos_list = tp_pos_list[0]
        
        # recip_rank = 1/rank
        recip_rank = 1 / float(first_pos_list)

    return recip_rank

def compute_mrr_at_k(eval_news_category_items:list):
    '''
    ## Purpose
    `compute_mrr_at_k()`: Computes the MRR (average RR) at cutoff k. In sum, it takes the mean average of all of the reciprocal rank scores among the actual vs. predicted labels. Review this ["Mean reciprocal rank" wikipedia article](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) for a simple explainer.
    ## Parameters
    - `eval_news_category_items` (List): List that contains 2 values
        1. String - Actual news genre category
        2. List of strings - Predicted news genre category in order by estimated probability to be returned by the model.
            - The example below shows how 
                - `'HEALTHY LIVING'` was the actual label, but it was third in 'reciprocal rank' with a value of 1/3
                - `'WORLDPOST'` was the actual label, and it was first in 'reciprocal rank' with a value of 1
                
                [
                    [
                        ['HEALTHY LIVING'], ['POLITICS', 'ENTERTAINMENT', 'HEALTHY LIVING']
                    ], 
                    [
                        ['WORLDPOST'], ['WORLDPOST', 'MEDIA', 'POLITICS']
                    ], 
                    ...
                ]

    ## Return Values
        - `mean_reciprocal_rank_score` (Float): Mean average reciprocal rank score among the predicted news category in the model
    '''
    rr_total = 0
    
    for item in eval_news_category_items:
        actual_label = item[0]
        pred_label_list = item[1]

        # Find the reciprocal rank (RR) for this row
        rr_at_k = _reciprocal_rank(actual_label, pred_label_list)

        # Add the row's RR to the accruing scores for the entire corpus
        rr_total = rr_total + rr_at_k

        # Update the Mean Reciprocal Rank (MRR) score with new row value
        mean_reciprocal_rank_score = rr_total / 1/float(len(eval_news_category_items))

    return mean_reciprocal_rank_score

def collect_preds(Y_test, Y_preds):
    '''
    ## Purpose
    Collect all predictions (predicted news genre labels) and ground truth (i.e., actual news genre label)
    '''
    pred_gold_list = [ [ [Y_test[index]], pred ] for index, pred in enumerate(Y_preds) ]
    return pred_gold_list
             
def compute_accuracy(eval_news_category_items:list):
    '''
    ## Purpose
    `compute_accuracy()`: Compute the overall accuracy score of the model across the training corpus

    ## Parameters
        - `eval_news_category_items` (List): List that contains 2 values
            1. String - Actual news genre category
            2. List of strings - Predicted news genre category

            Example: [
                [
                    ['HEALTHY LIVING'], ['POLITICS', 'ENTERTAINMENT', 'HEALTHY LIVING']
                ], 
                [
                    ['WORLDPOST'], ['WORLDPOST', 'MEDIA', 'POLITICS']
                ], 
                ...
            ]
    ## Return Values
        - `news_cat_prediction_accuracy` (Float): Percentage of accurately predicted news category in the model
    '''
    correct_news_cat = 0
    
    for news_genre_cat in eval_news_category_items:
        true_pred = news_genre_cat[0]
        machine_pred = set(news_genre_cat[1])
        
        for news_cat in true_pred:
            if news_cat in machine_pred:
                correct_news_cat += 1
                break
    
    news_cat_prediction_accuracy = correct_news_cat / float(len(eval_news_category_items))
    return news_cat_prediction_accuracy



In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def extract_features(df, field, training_data, testing_data, type='binary'):
    '''
    ## Purpose
    `extract_features()`: Extract features using different method types: binary, counts, or TF-IDF

    ### IF BINARY FEATURE REPRESENTATION
    Creates a new `CountVectorizer()` method object, which converts a collection of text documents to a matrix of token counts. Logistic regression involves vectorizing weighted averages of these tokens.
    '''
    
    logging.info("Extracting features and creating vocabulary...")

    '''
        BINARY and COUNTS PROCESSES WILL DO THE FOLLOWING:

        sklearn's CountVectorizer() will convert text to numerical data.
    '''
    
    if 'binary' in type:
        
        # BINARY FEATURE REPRESENTATION
        # Creates a new CountVectorizer() method object, which can help us use built-in functions that convert a collection of text documents to a matrix of token counts. **REMEMBER** that logistic regression involves vectorizing weighted averages of these tokens.
        # NOTE: `max_df` == "Maximum Document Frequency. It enables us to programmatically ignore frequently occuring words, e.g., articles like 'a' or 'the'. `max_df` reviews how many documents contain the word, and if it exceeds the max_df threshold then it is eliminated from the sparse matrix. Below we set the threshold to 95%.
        cv = CountVectorizer(binary=True, max_df=0.95)
        # CountVectorizer()'s fit_transform() uses the training_data to learn the vocabulary dictionary and return document-term matrix.
        cv.fit_transform(training_data[field].values)
        # CountVectorizer()'s transform() 
        train_feature_set = cv.transform(training_data[field].values)
        test_feature_set = cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
  
    elif 'counts' in type:
        
        # COUNT BASED FEATURE REPRESENTATION
        cv = CountVectorizer(binary=False, max_df=0.95)
        cv.fit_transform(training_data[field].values)
        
        train_feature_set = cv.transform(training_data[field].values)
        test_feature_set = cv.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,cv
    
    elif 'tfidf':    
        
        # TF-IDF BASED FEATURE REPRESENTATION
        tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)
        tfidf_vectorizer.fit_transform(training_data[field].values)
        
        train_feature_set=tfidf_vectorizer.transform(training_data[field].values)
        test_feature_set=tfidf_vectorizer.transform(testing_data[field].values)
        
        return train_feature_set,test_feature_set,tfidf_vectorizer

def get_top_k_predictions(model, X_test, k):
    '''
    ## Purpose
    `get_top_k_predictions()`: Uses the input trained LogisticRegression model to return the news genre class/category with the top estimated probability score.
    ## Parameters
    - `model` (LogisticRegression()): Trained model scikit-learn object
    - `X_test` (pandas DataFrame): Sampled test data set returned by `training_test_split()` in the `training_model()` function
    - `k` (Integer): Number of top categories (news genres) to return based on the estimated probability to predict the news genre
    ## Return Value(s)
    - `preds` (List of list): A list within a list of the top k retruned news categories. For example:
        - `preds` is `[['SCIENCE', 'HEALTHY LIVING', 'GREEN']]` for an article with the headline of `"Exercise in space keeps astronauts from fainting when they return to Earth, study says"` and `k=3`
    '''
    
    # get probabilities instead of predicted labels, since we want to collect top 3
    probs = model.predict_proba(X_test)

    # GET TOP K PREDICTIONS BY PROB - note these are just index
    best_n = np.argsort(probs, axis=1)[:,-k:]
    
    # GET CATEGORY OF PREDICTIONS
    preds = [[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    
    preds = [ item[::-1] for item in preds]
    
    return preds
   
def train_model(df, field="text_desc", feature_rep="binary", top_k=3):
    '''
    ## Purpose
    train_model() is the main controller function that conducts the following modeling procedure: 
        
    1. Create X data (List) by splitting the data to create two sampled sets: 1) for training, and 2) for testing.
    2. Create Y data (List) by assigning the actual (ground truth) labels
    3. Extract the features for the model to use, based on the chosen feature representation: binary vs. TF-IDF
    4. Fit, i.e., train, the logistic regression classifier model with scikit-learn's `LogisticRegression()` object
    5. Retrieve the evaluation items, e.g., the actual labels (ground truths) and predicted labels (list of top `k` number of estimated probable predicted categories)
    6. Use the evaluation iitems to compute the overall accuracy score and mean reciprocal rank score of the model

    ## Parameters
    - `df` (pandas DataFrame): the complete data set / corpus
    - `field` (String): the column name of the feature used to train the model
    - `feature_rep` (String): Type of LR analysis set as either "binary" or "count" or "tfidf"
    '''
    
    logging.info("Starting model training...")
    
    # 1. GET A TRAIN TEST SPLIT (set seed for consistent results)
    # train_test_split() from sklearn "splits arrays or matrices into random train and test subsets."
    # returns 2 new dataframes: one for training, another for testing the trained model
    y = df['category']
    x_training_data,x_testing_data = train_test_split(
        df,
        random_state=2000 #Controls the shuffling applied to the data before applying the split
    )

    # 2. GET LABELS FROM SPLIT DATA
    # Get the category values from each split data returned by #1
    Y_train = x_training_data['category'].values
    Y_test = x_testing_data['category'].values
     
    # 3. GET FEATURES
    X_train,X_test,feature_transformer = extract_features(
        df,
        field,
        x_training_data,
        x_testing_data,
        type=feature_rep
    )

    # INITIALIZE THE LOGISTIC REGRESSION CLASSIFIER OBJECT
    logging.info("Training a Logistic Regression Model. This may take a few minutes. ...")
    scikit_log_reg = LogisticRegression(
        verbose=0, #if you want the LR method to print out all the details, change this 0 to 1
        solver='liblinear',
        random_state=0,
        C=5,
        penalty='l2',
        max_iter=1000
    )
    # Create the model by providing the LR object the 
    model = scikit_log_reg.fit(X_train, Y_train)

    # GET TOP K PREDICTIONS
    preds = get_top_k_predictions(model, X_test, top_k)
    
    # GET PREDICTED VALUES AND GROUND TRUTH INTO A LIST OF LISTS - for ease of evaluation
    eval_items = collect_preds(Y_test, preds)
    
    # GET EVALUATION NUMBERS ON TEST SET -- HOW DID WE DO?
    logging.info("Starting evaluation...")
    simple_mean_avg_correct_prediction_accuracy = compute_accuracy(eval_items)
    mean_recip_rank_at_k = compute_mrr_at_k(eval_items)
    
    logging.info("Done training and evaluation.")

    # Return the herein computed model and other values for potential use and exploration
    return model,feature_transformer,simple_mean_avg_correct_prediction_accuracy,mean_recip_rank_at_k,X_train,X_test,Y_test,Y_train,preds,eval_items


### 4.4 LR Model 1 - Binary features with `text_desc` only

Train the model based on the short description data.

**FYI**. Your functions can return multiple variables, if you'd like. 

If you review the `train_model()` function, it returns multiple variables in a specific order. Those mirror the variables and the comma-separated variables and their desired types below.

#### 4.4.1 Enact the Training

In [ ]:
# Parameters to configure for our `train_model()` function
training_field = 'text_desc' #use the short description only to train a model
feature_rep = 'count' # Specify if this model should use a binary approach to the features (0 or 1) or the actual counts created by CountVectorizer()
top_k = 3 #tell the model function to return the top 3 'best fits' among the distributed probabilities

# Train that supervised ML logistic regression model!
model_td_only,transformer_td_only,accuracy_td_only,mrr_at_k_td_only,X_train,X_test,Y_test,Y_train,preds,eval_items = train_model(
  df, # full corpus
  field=training_field,
  feature_rep=feature_rep,
  top_k=top_k
)

#### Model Assessment Roadmap

In the following subsections, you will examine the LR model that you just trained by:

1. Print out the overall accuracy scores for the model
2. Compare the predictive power of the model across the news genre categories, i.e., *classes*
3. Visualize the predictive power per class/category with a "confusion matrix" heatmap
4. Dig deeper into the categories by comparing some categories' performance against the EDA work on the data set used to train the model

#### 4.4.2 Test the accuracy/performance of the model

##### 4.4.2.1 See the accuracy and Mean Reciprocal Rank Scores

We already computed the overall accuracy and MRR scores, when we trained the model, so let's output them.

If you need to refresh yourself on these scores, check out the functions' documentation in the 4.3 section.

In [ ]:
print(f"Simple Mean Average Model Accuracy = {accuracy_td_only}\nMean Reciprocal Rank = {mrr_at_k_td_only}")

##### 4.4.2.2 Compare Actual Labels (ground truths) with Predicted Labels (with the k threshold)

Before we test our model with headline inputs, we can test its performance with scikit-learn's `predict_proba()` associated function with the output model.

This function returns the estmated probability of each categorical label (news genre) in the test sample. With this returned data set, we can visualize the results to see what categories might be mislabeled more often than others and, of course, which categories are performing well.

Here is a description of parameter and return data:
- `X_test`: sample of the original data to test the trained model
- `Y_probability_td_only`: List of the predicted news genre category outputs (String)
- `Y_probability_a_td_only`: List of lists of the predicted news genre category based on the estimated probability score
  - First list == 1 row from X_test data set
    - Lists within that list == Each list contains estimated probability scores (Float) per (31) news genre categories

In [ ]:
# Predict the classes on the test data
Y_predictions_td_only = model_td_only.predict(X_test)
# Predict the classes on the test data, and return the probabilities for each class
Y_probability_a_td_only = model_td_only.predict_proba(X_test)

#### 4.4.3 Visualize Actual vs. Prediction with a Confusion Matrix

A confusion matrix helps you organize a direct comparison across all of your possible categories. Essentially, it asks: 

1. How many times did X (actual) equal the predicted (Y)?
2. How many times did X (actual) not equal the predicted (Y)?

Another way to phrase the questions is to think of it in action. So, with this data set, there are 31 possible categories (news genres), which means we have 31 categories to test against each other—**that's 961 categories** (31 * 31 = 961)!

A confusion matrix maps what are called the True Positive, False Positive, False Negative, and True Negative prediction outcomes. In the table below, I provide a high-level description of what each of these values represent in the scheme of our LR model on predicting news genres.

<div style="max-width:700px;">
  <table border="1">
  <tbody><tr>
    <td style="background:lightgreen" width="50%">
      <b>True Positive (TP):</b>
      <ul>
        <li>Reality: Input SCIENCE headline</li>
        <li>LR predicted: "SCIENCE"</li>
        <li>Outcome: LR's prediction is correct/accurate</li>
      </ul>
    </td>
    <td style="background:pink">
      <b>False Positive (FP):</b>
      <ul>
        <li>Reality: Input is <strong>NOT</strong> a SCIENCE headline</li>
        <li>LR predicted: "SCIENCE"</li>
        <li>Outcome: LR's prediction is incorrect/inaccurate</li>
    </ul></td>
  </tr>
  <tr>
    <td style="background:pink">
      <b>False Negative (FN):</b>
      <ul>
        <li>Reality: Input SCIENCE headline</li>
        <li>LR predicted: <strong>NOT</strong> "SCIENCE"</li>
        <li>Outcome: LR's prediction is incorrect/inaccurate</li>
      </ul>
    </td>
    <td style="background:lightgreen">
      <b>True Negative (TN):</b>
      <ul>
        <li>Reality: Input is <strong>NOT</strong> a SCIENCE headline</li>
        <li>LR predicted: <strong>NOT</strong> "SCIENCE"</li>
        <li>Outcome: LR's prediction is correct/accurate</li>
      </ul>
    </td>
  </tr>
</tbody></table></div>
</div>

Ok, so let's visualize the TPs, FPs, FNs, and TNs as a heatmap with annotated values for quick reference.

In [ ]:
cm_td_only = confusion_matrix(
  Y_test, #Sorted List of ground truth (correct/actual) target values
  Y_predictions_td_only #Sorted List of estimated targets as returned by a classifier
)

Let's be sure to **normalize the confusion matrix values**.

The current values in the confusion matrix are simply counts (Integers). But, if we want to visualize the prediction efficacy as a heatmap, we first need to "normalize" the values by converting them to a score relative to all of the other comparisons in the same column. For example, `ARTS` has results across 31 categories

In [ ]:
# Normalize matrix by columns
cm_td_only_normed_by_column = normalize(cm_td_only, axis=0, norm='l1')
# Compare the original to the normalized
print(
  'Simple Counts of the ARTS (first) column:\n', cm_td_only[0],'\n\n',
  'Normalized Counts of the ARTS (first) column:\n',cm_td_only_normed_by_column[0],
)

##### 4.4.3.2 Heatmap of the `text_desc` only normalized confusion matrix

In [ ]:
plt.figure(figsize=(31,14))

sns.heatmap(
  cm_td_only_normed_by_column, #normalized confusion matrix
  annot=True, # add normalized counts of co-occurrences between actual vs. predicted
  fmt=".3f", # round to thousandths decimal place
  linewidths=1, # style choice for row/column lines
  square=True, # make the 
  cmap='Purples',
  xticklabels=model_td_only.classes_,
  yticklabels=model_td_only.classes_,
)

# Label the X and Y axes
plt.ylabel('Actual Label of News Genre')
plt.xlabel('Predicted Label of News Genre')

# Let's plug in our overall accuracy measures into the the title
all_sample_title = f"Short Description Count Model\nOverall Accuracy Score: {accuracy_td_only}\nMean Reciprocal Rank: {mrr_at_k_td_only}"
plt.title(
  all_sample_title,
  size=15
)

# Ok, output time!
plt.tight_layout()
plt.show()

#### 4.4.4 Explore and Assess Categories/Classes with True Positive Rates, False Positive Rates, etc.

Based on the above overall accuracy score and heatmap results per category, you can begin to identify news genre categories (or classes) that may be performing well versus categories that are not performing well. You can use the following code below to explore those performance biases per news genre in the corpus/model.

As you work through the code below, take notes as you compare the overall accuracy score of the model with specific optimal threshold scores, and be sure to compare those results against the EDA work that you conducted earlier in the notebook. 

Our LR model has 31 classes (news genres), so it's a "multi-class" LR model. To help us explore the predictive accuracy across each class, we must first use scikit-learn's `LabelBinarizer()` to enable us to compute the True Positive Rates (TPR) and False Positive Rates (FPR) for each class/category.

The 3 variables assigned below will help us process the training and testing data to do so.

In [ ]:
label_binarizer = LabelBinarizer().fit(Y_train)
y_onehot_test = label_binarizer.transform(Y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
y_onehot_test

##### ROC Curve & AUC

An *ROC curve* (Receiver Operating Characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots the False Positive (x) and True Positive (y) rates at their different classification thresholds to help us visualize the performance of the model at different potential thresholds. In effect, this visual can help you isolate the performance of some classes over others in relationship with the original EDA work that you have conducted.

You can also use ROC curve plots to see how when you lower the classification threshold, the model will classify more items as positive and increase both False Positives and True Positives.

Also, AUC stands for Area Under the Curve. It aggregates all of the probable classification thresholds as a value between 0 and 1. 
- 0 == The model is wrong 100% of the time
- 1 == The model is correct 100% of the time

Think of it like the spectrum between 0 and 1 below, where the random positive predictions are positioned to the right of the random negative predictions. So, if the AUC score is `0.714`, then the probability that `p` is positioned to the right of `n` is `71.4%`.

```
Ouput of the LR Model
n = Actual Negative Prediction
p = Actual Positive Prediction

 nnnnnnnnnnnnnnnnnnnnnpnnnnpnnpnppnnpnppppppppppppp
|--------------------------------------------------|
0                                                  1
```

In [ ]:
def roc_curve_per_category(list_classes):
  '''
  ## Purpose
  `roc_curve_per_category`: Combine all of the ROC () values for each category in one DataFrame

  ## Parameters
  - `list_classes` (List): A list of the classes in the trained model

  ## Return Value(s)
  - `df_fpr_tpr` (pandas DataFrame): Dataframe with the following values per row:
    - `'Class'`: Class from the model for instance
    - `'FPR'`: False Positive Rate for instance
    - `'TPR'`: True Positive Rate for instance
    - `'Threshold_Value'`: Threshold Value for instance
    - `'Threshold_Optimal'`: Optimal threshold value for the class as a whole
    - `'GMean_Optimal'`: Optimal GMean value for the class as a whole
    - `'FPR_Optimal'`: Optimal FPR value for the class as a whole
    - `'TPR_Optimal'`: Optimal TPR value for the class as a whole
  '''
  list_dicts_classes_roc = []

  for class_cat in list_classes:
    # get class category (news genre)
    class_id = np.flatnonzero(label_binarizer.classes_ == class_cat)[0]
    Y_probability_a_td_only[:, class_id]

    fpr, tpr, thresholds = roc_curve(y_onehot_test[:, class_id], Y_probability_a_td_only[:, class_id])
    
    # Calculate the Geometric-Mean
    geometric_mean = np.sqrt(tpr * (1 - fpr))
    
    # Find the optimal threshold
    index = np.argmax(geometric_mean)
    threshold_optimal = round(thresholds[index], ndigits=4)
    gmean_optimal = round(geometric_mean[index], ndigits=4)
    fpr_optimal = round(fpr[index], ndigits=4)
    tpr_optimal = round(tpr[index], ndigits=4)

    for i in range(0, len(fpr)):
      list_dicts_classes_roc.append({
        'Class': class_cat,
        'FPR': fpr[i],
        'TPR': tpr[i],
        'Threshold_Value': thresholds[i],
        'Threshold_Optimal': threshold_optimal,
        'GMean_Optimal': gmean_optimal,
        'FPR_Optimal': fpr_optimal,
        'TPR_Optimal': tpr_optimal,
      })

  df_fpr_tpr = pd.DataFrame(list_dicts_classes_roc)

  return df_fpr_tpr

In [ ]:
df_all_classes_roc_values = roc_curve_per_category(model_td_only.classes_)
df_all_classes_roc_values.info()

Let's isolate the optimal values per class/category/news genre.

In [ ]:
df_all_classes_optimal_roc_values = df_all_classes_roc_values.drop_duplicates(subset=['Class']).reset_index()[['Class','Threshold_Optimal','GMean_Optimal','FPR_Optimal','TPR_Optimal']]

df_all_classes_optimal_roc_values

In [ ]:
type(Y_probability_a_td_only)

In [ ]:
def plot_class_roc_curve(class_of_interest, Y_one_vs_all, Y_prob_a, df_class_row):
    '''
    ## Purpose
    `plot_class_roc_curve`: Automate the visualizing of the FPR vs TPR of a particular class/category using the `RocCurveDisplay` function

    ## Parameters
    - `class_of_interest` (String): Category/class name to isolate
    - `Y_one_vs_all` (numpy array): List (array) of binarized values produced by `LabelBinarizer`'s `.transform()` on the Y_test data set.
        - NOTE: Function assumes that `label_binarizer` as an object with that name has been initialized and assigned as such.
    - `Y_prob_a` (numpy array): Array of probability estimates for each class/category.
    - `df_class_row` (row from pandas Dataframe): This dataframe row includes the optimal

    ## Return Values
    - None. Instead, it "shows" the matplotlib plot object.
    '''
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

    RocCurveDisplay.from_predictions(
        Y_one_vs_all[:, class_id],
        Y_prob_a[:, class_id],
        name=f"{class_of_interest} vs the rest",
        color="white",
        plot_chance_level=True,
    )

    # Plot best threshold
    # x = FPR_Optimal, y = TPR_Optimal
    plt.plot(
        df_class_row.FPR_Optimal, 
        df_class_row.TPR_Optimal,
        marker="o",
        markerfacecolor='white',
        markeredgecolor='black',
        markersize=10,
    )
    opt_x = df_class_row.FPR_Optimal
    opt_y = df_class_row.TPR_Optimal
    plt.annotate(
        f"Optimal threshold for {class_of_interest} ({str(opt_y.values.tolist()[0])})",
        (opt_x, opt_y), #x,y point to label
        xytext=(opt_x+0.03, opt_y-0.05)
    )

    plt.style.use('cyberpunk')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"One-vs-Rest ROC curves:\n{class_of_interest} vs Rest", color='white')
    plt.tight_layout()
    mplcyberpunk.add_glow_effects()
    plt.show()

In [ ]:
class_of_interest = "GREEN"
plot_class_roc_curve(
  class_of_interest=class_of_interest,
  Y_one_vs_all=y_onehot_test,
  Y_prob_a=Y_probability_a_td_only,
  df_class_row=df_all_classes_optimal_roc_values.loc[df_all_classes_optimal_roc_values.Class == class_of_interest]
)

#### 4.4.5 `text_desc` only & count-based modeling assessment

- **Accuracy**: 
- **MRR**:
- **Insert Observation #1**
  - Enter thoughts about this observation based on the available assessment evidence above
- **Insert Observation #2**
  - Enter thoughts about this observation based on the available assessment evidence above
- ...

### 4.5 Model 2 - `tfidf` features with `text_desc_headline` - short description + headline

In [ ]:
field='text_desc_headline'
feature_rep='tfidf'
top_k=3

model_tfidf_tdh,transformer_tfidf_tdh,accuracy_tfidf_tdh,mrr_at_k_tfidf_tdh,X_tfidf_tdh_train,X_tfidf_tdh_test,Y_tfidf_tdh_test,Y_tfidf_tdh_train,preds_tfidf_tdh,eval_items_tfidf_tdh = train_model(
  df,
  field=field,
  feature_rep=feature_rep,
  top_k=top_k
)

print(f"\n\nSimple Mean Average Model Accuracy = {accuracy_tfidf_tdh}\nMean Reciprocal Rank = {mrr_at_k_tfidf_tdh}\n")

#### 4.5.5 `text_desc`, `headline`, & TF-IDF-based modeling assessment

Now, use the same assessment techniques as in the `text_desc` only model to assess this model that uses TF-IDF + a combination of the short description and headline values as features for the model.

Write your observations and evidence below.

- **Accuracy**: 
- **MRR**:
- **Insert Observation #1**
  - Enter thoughts about this observation based on the available assessment evidence above
- **Insert Observation #2**
  - Enter thoughts about this observation based on the available assessment evidence above
- ...

### 4.6 Model 3 - `tfidf` features with `text_desc_headline_url`: description, headline, *and* url

In [ ]:
field='text_desc_headline_url'
feature_rep='tfidf'
top_k=3

model_tfidf_all,transformer_tfidf_all,accuracy_tfidf_all,mrr_at_k_tfidf_all,X_tfidf_all_train,X_tfidf_all_test,Y_tfidf_all_test,Y_tfidf_all_train,preds_tfidf_all,eval_items_tfidf_all = train_model(
  df,
  field=field,
  feature_rep=feature_rep,
  top_k=top_k
)

print("\nAccuracy={0}; MRR={1}".format(accuracy_tfidf_all,mrr_at_k_tfidf_all))

#### 4.6.5 `text_desc`, `headline`, `URL`, & TF-IDF-based modeling assessment

Now, use the same assessment techniques as the previous models to assess this third model that uses TF-IDF + a combination of the short description, headline, and URL values as features for the model.

Write your observations and evidence below.

- **Accuracy**: 
- **MRR**:
- **Insert Observation #1**
  - Enter thoughts about this observation based on the available assessment evidence above
- **Insert Observation #2**
  - Enter thoughts about this observation based on the available assessment evidence above
- ...

## 5. Check Predictions on Unseen Articles

Try from articles that aren't from HuffPost: CNN, Fox, MSNBC, etc.

Remember that the algorithm will return the top `k` probabilities. I've created a default of 3 for `k`. The first value in the list is the top returned predicted value.

### 5.1 Model 1 trained by counts method &amp; the features from the short description only

This uses Model 1: 

- Model: `model_td_only`
- Transformer: `transformer_td_only`

In [ ]:
# https://www.cnn.com/2019/07/19/health/astronaut-exercise-iv-faint-scn/index.html
test_features = transformer_td_only.transform(["Exercise in space keeps astronauts from fainting when they return to Earth, study says."])
get_top_k_predictions(
  model_td_only, 
  test_features,
  3
)

### 5.2 Model 2 trained by TF-IDF &amp; the features from the short description &amp; headline

This uses Model 2:

- Model: `model_tfidf_tdh`
- Transformer: `transformer_tfidf_tdh`

In [ ]:
# URL: https://www.network.com/enter/url/to/story/here.html

test_features=transformer_tfidf_tdh.transform(
    [
     "Enter headling here from story above"
    ]
)

get_top_k_predictions(
  model_tfidf_tdh,
  test_features,
  3
)

### 5.3 Model 3 trained by TF-IDF &amp; the features from the short description, headline &amp; URL

This uses Model 3:

- Model: `model_tfidf_all`
- Transformer: `transformer_tfidf_all`

In [ ]:
# URL: https://www.network.com/enter/url/to/story/here.html
test_features=transformer_tfidf_tdh.transform(
    [
     "Enter headling here from story above"
    ]
)

get_top_k_predictions(
  model_tfidf_tdh,
  test_features,
  3
)

## 6. Save Models for Future Use


In [ ]:
# Use imported `pickle` code library

#### 1. COUNTS - 'text_desc' as features only ####
model_path_td_only = "./../models/news_genre_model_binary_text_desc_only.pkl"
transformer_path_td_only = "./../models/news_genre_transformer_binary_text_desc_only.pkl"
# Save both the transformer -> to encode a document and the model itself to make predictions based on the weight vectors 
pickle.dump( model_td_only, open(model_path_td_only, 'wb') )
pickle.dump( transformer_td_only, open(transformer_path_td_only,'wb') )

#### 2. TF-IDF - 'text_desc_headline' as features ####
model_path_tdh = "./../models/news_genre_model_tfidf_tdh.pkl"
transformer_path_tdh = "./../models/news_genre_transformer_tfidf_tdh.pkl"
# Save both the transformer -> to encode a document and the model itself to make predictions based on the weight vectors 
pickle.dump( model_tfidf_tdh, open(model_path_tdh, 'wb') )
pickle.dump( transformer_tfidf_tdh, open(transformer_path_tdh,'wb') )

#### 3. TF-IDF - 'text_desc_headline_url' as features ####
model_path_all = "./../models/news_genre_model_tfidf_all.pkl"
transformer_path_all = "./../models/news_genre_transformer_tfidf_all.pkl"
# Save both the transformer -> to encode a document and the model itself to make predictions based on the weight vectors 
pickle.dump( model_tfidf_all, open(model_path_all, 'wb') )
pickle.dump( transformer_tfidf_all, open(transformer_path_all,'wb') )


## 7. Use Loaded Model

With pickle, we can now also load the model and use it without needing to retrain everything. A good time saver to consider for your team's final project!

In [ ]:
# Load Model 2 for reference
# Note my variable assignments include "loaded_" so I can distinguish variable assignments, if necessary.

model_path_tdh = "./../models/news_genre_model_tfidf_tdh.pkl"
transformer_path_tdh = "./../models/news_genre_transformer_tfidf_tdh.pkl"

loaded_model_tfidf_tdh = pickle.load(open(model_path_tdh, 'rb'))
loaded_transformer_tfidf_tdh = pickle.load(open(transformer_path_tdh, 'rb'))

# URL: https://www.network.com/enter/url/to/story/here.html
loaded_test_features_tfidf_tdh = loaded_transformer_tfidf_tdh.transform(["Enter headling here from story above"])
get_top_k_predictions(loaded_model_tfidf_tdh, loaded_test_features_tfidf_tdh, 3)


## Conclusion

Overall, do your best to have understood this LR modeling/training process: the goals for the model in relationship to the original data set used. By documenting your observations and using Python to conduct EDA + model assessments, you can complete the following work that you will need to repeat for your final project:

1. Identify potential boundaries, biases, and limits of the data in relationship to your developing and changing goals for modeling
2. Conversely, identify potential possibilities and affordances of the data in relationship to your developing and changing goals for modeling
3. Documenting these biases, affrodances, and changes in goals
4. Use this notebook material to develop a Model Card that communicates the aforementioned material in a more concise and helpful manner